In [1]:
import pandas as pd
import multiprocessing
import numpy as np
import time
from omilayers import Omilayers
import io

# Connect to database

In [2]:
omi = Omilayers("project.duckdb")

# Helper functions

In [2]:
Nsamples = 100
sampleIDs = ["SA"+f"{i}".zfill(3) for i in range(1, Nsamples+1)]

def timeit(func):
    """
    Decorator that times the execution of a function.
    """
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        duration = np.round(end_time - start_time, 2)
        return (duration, result)
    
    return wrapper

@timeit
def simulate_data(featuresName, features, minValue, maxValue, integers=False):
    """
    Function for simulating data.
    """
    JSON = {}
    for feature in features:
        if not integers:
            JSON[feature] = np.round(np.random.uniform(minValue, maxValue, Nsamples), 4)
        else:
            JSON[feature] = np.random.randint(minValue, maxValue, Nsamples)
    data = pd.DataFrame(JSON, index=sampleIDs)
    data = data.T
    data = data.reset_index()
    data = data.rename(columns={"index":featuresName})
    
    return data

def memory_usage_simulated_data(data):
    """
    Function for estimating the memory usage of a pandas.DataFrame.
    """
    output = io.StringIO()
    data.info(memory_usage='deep', buf=output)
    output = output.getvalue().split("\n")
    memoryUsage = output[-2].split(":")[-1].strip()
    
    return memoryUsage

@timeit
def store_data(dataToStore, layerName, tagName, description):
    """
    Function for storing omic layer data.
    """
    omi.layers[layerName] = "dataToStore"
    omi.layers[layerName].set_tag(tagName)
    omi.layers[layerName].set_info(description)
    
    return None

@timeit
def load_layer(layerName):
    """
    Function for loading stored omic layer data.
    """
    layerData = omi.layers[layerName].to_df()
    
    return layerData

@timeit
def get_sample(layer, sample):
    result = omi.layers[layer][sample]
    
    return result

@timeit
def add_sample(layer, sampleName, data):
    omi.layers[layer][sampleName] = data
    
    return None



# Cohort features

## Create synthetic data

In [4]:
data = pd.DataFrame({
    "sample_id": sampleIDs,
    "gender": np.random.choice(["female", "male"], Nsamples),
    "age": np.random.randint(20, 50, Nsamples),
    "bmi": np.round(np.random.uniform(20, 40, Nsamples), 2)
})

Get memory usage of simulated data

In [5]:
memoryUsage = memory_usage_simulated_data(data)

In [6]:
print(f"Memory usage of simulated data: {memoryUsage}")

Memory usage of simulated data: 13.8 KB


## Store synthetic data

In [7]:
duration, _ = store_data(data, "cohort", "raw", "Cohort features.")

In [8]:
print(f"Time to store simulated data: {duration} s")

Time to store simulated data: 0.07 s


## Retrieve stored data

In [12]:
duration, layerData = load_layer("cohort")

In [13]:
print(f"Time to load stored layer: {duration} s")

Time to load stored layer: 0.02 s


In [14]:
layerData

,sample_id,gender,age,bmi
0,SA001,female,42,31.47
1,SA002,male,41,21.97
2,SA003,male,32,29.82
3,SA004,female,36,30.54
4,SA005,male,42,34.72
...,...,...,...,...
95,SA096,male,41,29.65
96,SA097,male,25,34.08
97,SA098,male,25,29.27
98,SA099,female,36,31.45


# Blood metabolomic data

## Create synthetic data

Load features

In [16]:
features = pd.read_csv("blood_metabolites-2024-06-25.csv")['NAME']
print(f"Number of features: {len(features)}")

Number of features: 37228


In [19]:
duration, data = simulate_data("metabolite", features, minValue=0, maxValue=1000, integers=False)

In [20]:
print(f"Time to create simulated data: {duration} s")

Time to create simulated data: 0.48 s


Get memory usage of simulated data

In [21]:
memoryUsage = memory_usage_simulated_data(data)

In [22]:
print(f"Memory usage of simulated data: {memoryUsage}")

Memory usage of simulated data: 31.6 MB


## Store synthetic data

In [23]:
duration, _ = store_data(data, "blood_metas", "raw", "Raw blood metabolomic data.")

In [24]:
print(f"Time to store simulated data: {duration} s")

Time to store simulated data: 1.24 s


## Retrieve stored data

In [25]:
duration, layerData = load_layer("blood_metas")

In [26]:
print(f"Time to load stored layer: {duration} s")

Time to load stored layer: 0.09 s


In [27]:
layerData

,metabolite,SA001,SA002,SA003,SA004,SA005,SA006,SA007,SA008,SA009,...,SA091,SA092,SA093,SA094,SA095,SA096,SA097,SA098,SA099,SA100
0,1-Methylhistidine,790.4385,479.1370,465.4743,255.7633,450.6971,176.2241,768.7096,969.0874,434.7642,...,676.5417,650.8838,544.7083,681.2309,729.1389,445.8075,791.4020,885.9607,422.0507,52.3249
1,"1,3-Diaminopropane",933.0550,596.9813,785.5830,819.0367,437.4976,377.9883,767.9444,909.9663,972.9616,...,822.3848,580.3374,702.6395,586.4949,921.2759,734.2608,887.1223,565.5894,819.2684,539.9147
2,2-Ketobutyric acid,779.2190,945.5954,325.1594,13.7247,891.7784,368.2823,478.2548,0.0550,876.3774,...,614.7638,470.3742,409.7030,508.5735,264.2065,453.4717,158.0714,71.2529,367.1318,456.2037
3,2-Hydroxybutyric acid,564.6894,966.6212,911.2661,160.9762,834.9034,996.4355,32.7219,391.2687,781.6500,...,641.3102,219.6839,614.7418,369.7123,873.3008,996.4800,532.6586,375.2889,192.1418,886.9793
4,2-Methoxyestrone,993.7184,945.3338,33.5006,186.7108,365.8788,169.4356,226.6432,256.0707,256.5543,...,477.0651,981.3518,600.1203,548.4905,44.8454,286.1296,203.6216,503.1338,959.2261,798.4632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37223,Succinylserine,426.3019,79.3807,105.2245,729.2262,144.6969,444.0021,114.5421,964.2886,183.6636,...,237.5009,394.1744,664.8537,847.1231,968.7375,491.2920,130.5776,490.9311,428.4874,659.0267
37224,4-Hydroxylidocaine,718.7427,310.3886,63.2965,746.0249,736.1797,183.4879,348.3069,311.4648,229.8678,...,898.9060,694.9580,820.5873,619.9116,976.5609,808.4757,534.3990,232.3339,433.6389,313.7139
37225,Tryptophan N-glucoside,763.2851,392.5069,333.6453,442.6015,679.5470,853.9085,562.4418,940.8951,294.8120,...,460.2327,247.3520,900.9480,153.3615,401.8935,383.3765,611.9506,123.0550,161.6592,702.3039
37226,"6-Amino-5-formamido-1,3-dimethyluracil",564.9067,386.0231,731.6503,466.3171,627.5026,258.3179,686.4799,246.8360,265.7690,...,111.0318,729.6773,543.0595,646.0449,82.8429,783.8854,380.4824,145.0220,945.2639,743.7019


# Urine metabolomic data

## Create synthetic data

Load features

In [28]:
features = pd.read_csv("urine_metabolites-2024-06-25.csv")['NAME']
print(f"Number of features: {len(features)}")

Number of features: 5661


In [29]:
duration, data = simulate_data("metabolite", features, minValue=0, maxValue=1000, integers=False)

In [30]:
print(f"Time to create simulated data: {duration} s")

Time to create simulated data: 0.1 s


Get memory usage of simulated data

In [31]:
memoryUsage = memory_usage_simulated_data(data)

In [32]:
print(f"Memory usage of simulated data: {memoryUsage}")

Memory usage of simulated data: 4.7 MB


## Store synthetic data

In [33]:
duration, _ = store_data(data, "urine_metas", "raw", "Raw urine metabolomic data.")

In [34]:
print(f"Time to store simulated data: {duration} s")

Time to store simulated data: 1.11 s


## Retrieve stored data

In [35]:
duration, layerData = load_layer("urine_metas")

In [36]:
print(f"Time to load stored layer: {duration} s")

Time to load stored layer: 0.07 s


In [37]:
layerData

,metabolite,SA001,SA002,SA003,SA004,SA005,SA006,SA007,SA008,SA009,...,SA091,SA092,SA093,SA094,SA095,SA096,SA097,SA098,SA099,SA100
0,1-Methylhistidine,520.9914,773.8543,886.9167,818.8746,739.5974,912.9197,36.4682,507.7532,640.2385,...,710.5847,3.3084,989.9201,528.6773,805.4751,741.7653,680.5614,186.4169,4.1461,223.4690
1,"1,3-Diaminopropane",794.1838,108.2707,877.6342,133.5558,228.9019,696.1183,694.9589,520.9567,929.4043,...,57.2923,71.0374,369.9070,188.3528,495.9905,432.4917,692.6938,595.2354,883.6898,101.0429
2,2-Ketobutyric acid,133.4277,127.2726,365.6887,581.9479,57.9929,626.7420,240.6875,438.9613,245.5784,...,399.2345,415.9058,176.1171,520.8446,879.4136,785.7176,665.0320,705.1218,209.6178,983.8270
3,2-Hydroxybutyric acid,758.4010,908.8080,446.6645,307.8894,130.6359,871.3185,757.4312,127.3789,140.5263,...,712.3591,780.1865,34.1524,339.5172,686.8037,23.4492,890.4642,951.3504,213.9050,758.8808
4,2-Methoxyestrone,325.3456,732.4450,950.1547,199.9093,213.2191,528.1664,645.0592,610.1322,144.7181,...,201.3537,270.9674,546.0308,778.0867,49.8483,165.0221,278.7912,138.6843,291.0290,503.9690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5656,Succinylserine,67.9395,670.0967,608.3988,835.2662,868.6693,793.3685,624.6668,488.2755,754.5705,...,867.0585,462.4128,459.4015,737.8237,19.7071,225.3894,585.9125,925.2182,268.0463,287.6336
5657,4-Hydroxylidocaine,181.9414,913.5699,621.0165,217.8846,650.0279,811.4643,412.1920,547.6706,265.7782,...,862.1512,784.7165,700.6826,977.6106,122.3832,708.6793,154.8254,220.6026,872.4163,919.0863
5658,Tryptophan N-glucoside,372.7239,560.6989,876.2181,640.2097,594.5999,818.3350,117.2741,44.8309,390.3994,...,222.0776,156.2529,655.0262,442.1350,419.6910,848.0367,911.9151,605.4445,955.6774,55.6342
5659,"6-Amino-5-formamido-1,3-dimethyluracil",673.8078,388.0446,657.4035,816.7510,115.6600,22.4933,684.9080,217.1122,925.3388,...,126.3337,989.6082,812.4913,366.3966,680.2468,692.4485,865.0907,979.6344,154.7754,317.1179


# Bulk RNASeq data

## Create synthetic data

Load features

In [38]:
features = open("hg38_ensembl_ids.txt").read().splitlines()
print(f"Number of features: {len(features)}")

Number of features: 60649


In [39]:
duration, data = simulate_data("gene", features, minValue=0, maxValue=1000, integers=True)

In [40]:
print(f"Time to create simulated data: {duration} s")

Time to create simulated data: 0.84 s


Get memory usage of simulated data

In [41]:
memoryUsage = memory_usage_simulated_data(data)

In [42]:
print(f"Memory usage of simulated data: {memoryUsage}")

Memory usage of simulated data: 50.4 MB


## Store synthetic data

In [43]:
duration, _ = store_data(data, "rnaseq", "raw", "Raw bulk RNASeq data.")

In [44]:
print(f"Time to store simulated data: {duration} s")

Time to store simulated data: 1.21 s


## Retrieve stored data

In [45]:
duration, layerData = load_layer("rnaseq")

In [46]:
print(f"Time to load stored layer: {duration} s")

Time to load stored layer: 0.13 s


In [47]:
layerData

,gene,SA001,SA002,SA003,SA004,SA005,SA006,SA007,SA008,SA009,...,SA091,SA092,SA093,SA094,SA095,SA096,SA097,SA098,SA099,SA100
0,ENSG00000223972,885,5,917,802,670,260,740,492,529,...,0,780,635,643,996,334,905,814,593,386
1,ENSG00000227232,899,55,611,266,67,247,472,635,769,...,710,656,899,467,671,665,573,835,938,639
2,ENSG00000278267,640,718,852,896,85,975,831,323,695,...,901,549,294,519,513,947,159,860,98,212
3,ENSG00000243485,73,924,96,501,956,695,721,923,891,...,794,563,945,19,51,752,396,540,588,964
4,ENSG00000284332,359,598,466,798,573,203,116,982,252,...,769,343,77,192,723,131,799,220,586,448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60600,ENSG00000198695,856,977,203,195,221,588,615,70,883,...,142,918,582,954,343,370,633,554,870,505
60601,ENSG00000210194,329,443,780,373,701,116,768,473,408,...,11,764,188,551,945,881,998,206,357,348
60602,ENSG00000198727,6,422,911,936,424,11,822,949,229,...,499,474,803,160,997,445,181,18,506,703
60603,ENSG00000210195,525,599,947,896,780,235,677,151,955,...,431,435,659,735,565,770,637,658,135,4


## Retrieve all stored features for sample

In [48]:
duration, sample = get_sample("rnaseq", "SA090")

In [49]:
print(f"Time to load sample with {len(sample)} features: {duration} s")

Time to load sample with 60605 features: 0.03 s


In [50]:
print(f"Memory footprint of data: {memory_usage_simulated_data(pd.Series(sample))}")

Memory footprint of data: 473.6 KB


## Add new sample

In [51]:
duration, _ = add_sample("rnaseq", "SA101", sample)

In [52]:
print(f"Time to add new sample with {len(sample)} features: {duration} s")

Time to add new sample with 60605 features: 127.38 s


# Gut microbiome data

## Create synthetic data

Load features

In [53]:
features = open("species.txt").read().splitlines()
print(f"Number of features: {len(features)}")

Number of features: 6914


In [54]:
duration, data = simulate_data("species", features, minValue=0, maxValue=1000, integers=True)

In [55]:
print(f"Time to create simulated data: {duration} s")

Time to create simulated data: 0.11 s


Get memory usage of simulated data

In [56]:
memoryUsage = memory_usage_simulated_data(data)

In [57]:
print(f"Memory usage of simulated data: {memoryUsage}")

Memory usage of simulated data: 5.8 MB


## Store synthetic data

In [58]:
duration, _ = store_data(data, "microbiome", "raw", "Raw gut microbiome data.")

In [59]:
print(f"Time to store simulated data: {duration} s")

Time to store simulated data: 1.25 s


## Retrieve stored data

In [60]:
duration, layerData = load_layer("microbiome")

In [61]:
print(f"Time to load stored layer: {duration} s")

Time to load stored layer: 0.07 s


In [62]:
layerData

,species,SA001,SA002,SA003,SA004,SA005,SA006,SA007,SA008,SA009,...,SA091,SA092,SA093,SA094,SA095,SA096,SA097,SA098,SA099,SA100
0,Bacteroides uniformis,90,780,50,605,157,166,636,906,420,...,366,725,682,19,254,387,329,601,140,914
1,Bacteroides ovatus,607,599,173,615,305,752,559,415,566,...,53,86,100,542,132,228,926,298,980,545
2,Bacteroides vulgatus,411,770,270,476,512,983,306,837,438,...,498,275,506,511,736,37,781,766,490,788
3,Blautia obeum,345,540,56,661,269,368,536,787,630,...,104,362,715,603,64,859,764,669,660,381
4,[Eubacterium] rectale,173,586,239,679,571,507,258,487,611,...,922,908,987,712,597,142,384,883,379,283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6909,Rathayibacter festucae,119,627,736,487,716,410,923,402,630,...,905,852,750,80,840,399,29,913,952,468
6910,Thermosipho sp. DSM 6568,641,356,284,155,512,673,749,32,510,...,115,688,744,718,413,142,67,673,74,823
6911,Demequina globuliformis,712,112,125,947,307,167,701,734,157,...,158,107,663,796,244,243,777,739,873,676
6912,Streptomyces abyssalis,988,612,332,454,922,55,645,525,964,...,915,409,89,434,164,25,416,639,820,190


# Genomic data (VCF)

## Download data

The synthetic VCF file is hosted on Zenodo where is can be downloaded.

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.12790872.svg)](https://doi.org/10.5281/zenodo.12790872)

## Store synthetic data

In [60]:
@timeit
def store_vcf(filename):
    omi.layers.from_csv(layer='vcf', filename=filename, sep='\t', chunksize=100000)
    return None

duration, _ = store_vcf("simulated.vcf.gz")

In [61]:
print(f"Time to store simulated data: {duration} s")

Time to store simulated data: 1726.73 s


Add tag and description to layer

In [62]:
omi.layers['vcf'].set_tag("raw")
omi.layers['vcf'].set_info("Cohort imputed VCF.")

## Add new sample

<span style='color:red'>**NOTE**</span><br>
The following line of code requires an **extensive runtime**.

In [ ]:
duration, _ = add_sample("vcf", "SA101", sample)

<span style='color:red'>**NOTE**</span> <br>
The execution of the above line of code was not allowed to finish during testing due to the extensive runtime needed. The execution was manually terminated after 1 hour of runtime. Below, the number of entries that were registered in the database during the 1 hour of execution is given.

In [23]:
print("Total number of entries inserted within 1 hour of execution using 8 threads: {} entries".format((omi.layers['vcf']['SA101'] != None).sum()))

Total number of entries inserted within 1 hour of execution using 8 threads: 312049 entries


Based on the above information, the addition of the whole sample with 9640953 features would require approximately 30 hours of runtime.

## Retrieve stored data

VCF includes character "#" infront of the 'CHROM" column. Rename column to remove character.

In [63]:
omi.layers['vcf'].rename("'#CHROM'", 'CHROM')

In [13]:
@timeit
def parse_vcf(chromo, pos, columns):
    if chromo is not None:
        result = omi.layers['vcf'].query(f"CHROM == '{chromo}' and POS == '{pos}'")[columns]
    else:
        result = omi.layers['vcf'].select(cols=columns, where='POS', values=pos)
    return result

@timeit
def parse_range_vcf(chromo, start, end, columns):
    result = omi.layers['vcf'].query(f"CHROM == '{chromo}' and POS BETWEEN {start} AND {end}")[columns]
    return result
    

In [5]:
duration, result = parse_vcf('chr3', 100000, ['ID', 'SA010', 'SA090'])
result     

,ID,SA010,SA090
rowid,,,
1654040,3:100000:G:T,"1|0:0.184:0.051,0.062:0.353,0.285,0.362","0|1:0.363:0.761,0.894:0.995,0.002,0.003"


In [6]:
print(f"Time to parse vcf: {duration} s")  

Time to parse vcf: 0.13 s


In [7]:
duration, result = parse_vcf('chr22', 100000, ['ID', 'SA010', 'SA090'])
result

,ID,SA010,SA090
rowid,,,
9588773,22:100000:T:A,"1|1:0.542:0.438,0.166:0.957,0.016,0.027","1|0:0.888:0.686,0.946:0.231,0.194,0.575"


In [8]:
print(f"Time to parse vcf: {duration} s")

Time to parse vcf: 0.17 s


In [9]:
duration, result = parse_vcf(chromo=None, pos=100000, columns=['ID', 'SA010', 'SA090'])
result

,POS,ID,SA010,SA090
rowid,,,,
90000,100000,1:100000:G:A,"0|1:0.641:0.88,0.203:0.934,0.052,0.014","0|1:0.322:0.518,0.689:0.519,0.382,0.099"
832932,100000,2:100000:C:T,"1|0:0.144:0.462,0.846:0.759,0.197,0.044","1|0:0.72:0.378,0.539:0.059,0.664,0.277"
1654040,100000,3:100000:G:T,"1|0:0.184:0.051,0.062:0.353,0.285,0.362","0|1:0.363:0.761,0.894:0.995,0.002,0.003"
2355176,100000,4:100000:C:G,"1|0:0.363:0.021,0.732:0.172,0.233,0.595","1|0:0.119:0.441,0.84:0.413,0.083,0.504"
3066329,100000,5:100000:C:G,"0|0:0.423:0.948,0.166:0.015,0.454,0.531","0|0:0.62:0.254,0.99:0.903,0.049,0.048"
3702775,100000,6:100000:C:A,"1|1:0.17:0.174,0.625:0.935,0.006,0.059","1|0:0.07:0.751,0.039:0.772,0.095,0.133"
4349834,100000,7:100000:A:C,"1|1:0.498:0.045,0.372:0.818,0.081,0.101","0|1:0.078:0.716,0.819:0.151,0.258,0.591"
4911808,100000,8:100000:T:C,"0|1:0.757:0.785,0.161:0.778,0.207,0.015","1|0:0.185:0.632,0.472:0.83,0.133,0.037"
5459234,100000,9:100000:A:C,"0|1:0.774:0.915,0.858:0.975,0.003,0.022","1|0:0.732:0.647,0.803:0.607,0.005,0.388"


In [10]:
print(f"Time to parse vcf: {duration} s")

Time to parse vcf: 0.1 s


In [14]:
duration, result = parse_range_vcf('chr15', 50000, 50010, ['ID', 'SA010', 'SA090'])
result

,ID,SA010,SA090
rowid,,,
7937317,15:50000:A:G,"1|1:0.502:0.24,0.938:0.434,0.19,0.376","1|0:0.975:0.318,0.538:0.176,0.674,0.15"
7937318,15:50001:C:T,"1|1:0.214:0.732,0.514:0.236,0.158,0.606","0|0:0.732:0.979,0.842:0.1,0.066,0.834"
7937319,15:50002:A:G,"1|1:0.396:0.537,0.581:0.534,0.277,0.189","0|0:0.446:0.276,0.065:0.747,0.047,0.206"
7937320,15:50003:A:G,"0|0:0.677:0.191,0.832:0.589,0.254,0.157","0|1:0.336:0.97,0.848:0.737,0.194,0.069"
7937321,15:50004:A:G,"1|0:0.947:0.946,0.091:0.468,0.477,0.055","0|1:0.252:0.284,0.098:0.888,0.037,0.075"
7937322,15:50005:T:G,"1|0:0.055:0.06,0.958:0.846,0.102,0.052","1|0:0.344:0.956,0.419:0.204,0.495,0.301"
7937323,15:50006:C:A,"0|0:0.108:0.658,0.017:0.771,0.056,0.173","0|1:0.85:0.109,0.291:0.932,0.028,0.04"
7937324,15:50007:T:C,"1|1:0.53:0.347,0.236:0.381,0.571,0.048","1|1:0.764:0.96,0.585:0.152,0.184,0.664"
7937325,15:50008:T:A,"1|1:0.542:0.539,0.925:0.344,0.492,0.164","1|1:0.799:0.346,0.185:0.125,0.456,0.419"


In [15]:
print(f"Time to parse vcf: {duration} s")

Time to parse vcf: 0.08 s


## Retrieve all stored features for sample

In [73]:
duration, sample = get_sample("vcf", "SA090")

In [74]:
print(f"Time to load sample with {len(sample)} features: {duration} s")

Time to load sample with 9640953 features: 1.88 s


In [75]:
print(f"Memory footprint of data: {memory_usage_simulated_data(pd.Series(sample))}")

Memory footprint of data: 876.5 MB


## Retrieve stored data in parallel

For parallel data retrieval, the database should be accessed in read-only mode

In [3]:
omi = Omilayers("project.duckdb", read_only=True)

In [4]:
def getSamples(chromo, pos):
    result = omi.layers['vcf'].query(f"CHROM == '{chromo}' and POS == {pos}")[['SA096', 'SA098', 'SA099']]
    return (chromo, pos, result.values)

@timeit
def queryInParallel(queries):
    with multiprocessing.Pool(processes=8) as pool:
        results = pool.starmap(getSamples, queries)
    return results

queries = []
for i in [1,10,20]:
    for j in range(1, 100):
        queries.append((f"chr{i}", 100000+j))
        
duration, results = queryInParallel(queries)

In [6]:
results[:5]

[('chr1',
  100001,
  array([['0|0:0.508:0.702,0.432:0.962,0.013,0.025',
          '1|1:0.913:0.092,0.546:0.795,0.028,0.177',
          '1|0:0.721:0.193,0.479:0.051,0.172,0.777']], dtype=object)),
 ('chr1',
  100002,
  array([['1|0:0.25:0.389,0.383:0.668,0.059,0.273',
          '0|1:0.681:0.042,0.261:0.72,0.034,0.246',
          '0|0:0.455:0.177,0.445:0.776,0.003,0.221']], dtype=object)),
 ('chr1',
  100003,
  array([['0|1:0.981:0.986,0.122:0.157,0.182,0.661',
          '1|0:0.513:0.84,0.457:0.292,0.133,0.575',
          '1|1:0.766:0.471,0.88:0.145,0.423,0.432']], dtype=object)),
 ('chr1',
  100004,
  array([['0|1:0.676:0.346,0.759:0.183,0.637,0.18',
          '1|0:0.061:0.735,0.559:0.236,0.309,0.455',
          '1|1:0.895:0.97,0.068:0.275,0.665,0.06']], dtype=object)),
 ('chr1',
  100005,
  array([['1|1:0.364:0.393,0.319:0.193,0.313,0.494',
          '0|1:0.569:0.968,0.188:0.516,0.027,0.457',
          '0|1:0.896:0.018,0.915:0.057,0.866,0.077']], dtype=object))]

In [7]:
print(f"Time to parse vcf in parallel: {duration} s")

Time to parse vcf in parallel: 6.43 s


# View stored omic layers

In [8]:
omi.layers

       name tag       shape                        info
     cohort raw       100x4            Cohort features.
        vcf raw 9640953x109         Cohort imputed VCF.
blood_metas raw   37228x101 Raw blood metabolomic data.
urine_metas raw    5661x101 Raw urine metabolomic data.
     rnaseq raw   60605x102       Raw bulk RNASeq data.
 microbiome raw    6914x101    Raw gut microbiome data.